# ML Zoomcamp 2025 - Homework 8

Hair type classification with a small CNN in PyTorch.

Follow the steps in this notebook to train the model and compute the quantities needed to answer questions 1–6 in `homework.md`.


In [4]:
import os
from pathlib import Path

import numpy as np
import torch
from torch import nn, optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

SEED = 42
np.random.seed(SEED)
torch.manual_seed(SEED)

if torch.cuda.is_available():
    torch.cuda.manual_seed(SEED)
    torch.cuda.manual_seed_all(SEED)

torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device


device(type='cpu')

In [5]:
data_root = Path("data")

if not data_root.exists():
    print("Downloading and extracting hair type dataset...")
    !wget https://github.com/SVizor42/ML_Zoomcamp/releases/download/straight-curly-data/data.zip -O data.zip
    !unzip -q data.zip
else:
    print(f"Found existing data at {data_root.resolve()}")

train_dir = data_root / "train"
test_dir = data_root / "test"
train_dir, test_dir


--2025-11-26 00:23:40--  https://github.com/SVizor42/ML_Zoomcamp/releases/download/straight-curly-data/data.zip
A resolver github.com (github.com)... 140.82.121.4
A ligar a github.com (github.com)|140.82.121.4|:443... ligado.
Pedido HTTP enviado, a aguardar resposta... 302 Found
Localização: https://release-assets.githubusercontent.com/github-production-release-asset/405934815/e712cf72-f851-44e0-9c05-e711624af985?sp=r&sv=2018-11-09&sr=b&spr=https&se=2025-11-26T01%3A21%3A25Z&rscd=attachment%3B+filename%3Ddata.zip&rsct=application%2Foctet-stream&skoid=96c2d410-5711-43a1-aedd-ab1947aa7ab0&sktid=398a6654-997b-47e9-b12b-9515b896b4de&skt=2025-11-26T00%3A21%3A00Z&ske=2025-11-26T01%3A21%3A25Z&sks=b&skv=2018-11-09&sig=L4cXsNmLSWgiFlrrCUrA3Te80lmYvRvyDVPuzPExxP8%3D&jwt=eyJ0eXAiOiJKV1QiLCJhbGciOiJIUzI1NiJ9.eyJpc3MiOiJnaXRodWIuY29tIiwiYXVkIjoicmVsZWFzZS1hc3NldHMuZ2l0aHVidXNlcmNvbnRlbnQuY29tIiwia2V5Ijoia2V5MSIsImV4cCI6MTc2NDExODQyMSwibmJmIjoxNzY0MTE2NjIxLCJwYXRoIjoicmVsZWFzZWFzc2V0cHJvZHVjdGlvbi5ib

(PosixPath('data/train'), PosixPath('data/test'))

In [6]:
image_size = (200, 200)
batch_size = 20

train_transforms = transforms.Compose([
    transforms.Resize(image_size),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    ),
])

test_transforms = transforms.Compose([
    transforms.Resize(image_size),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    ),
])

train_dataset = datasets.ImageFolder(train_dir, transform=train_transforms)
test_dataset = datasets.ImageFolder(test_dir, transform=test_transforms)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
validation_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

validation_dataset = test_dataset
class_names = train_dataset.classes
class_names


['curly', 'straight']

In [7]:
class HairTypeCNN(nn.Module):
    def __init__(self):
        super().__init__()
        # Input shape: (3, 200, 200)
        self.conv = nn.Conv2d(in_channels=3, out_channels=32, kernel_size=(3, 3))
        self.pool = nn.MaxPool2d(kernel_size=(2, 2))
        self.flatten = nn.Flatten()
        # After conv + pool: (32, 99, 99) -> 32 * 99 * 99 features
        self.fc1 = nn.Linear(32 * 99 * 99, 64)
        self.fc2 = nn.Linear(64, 1)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.conv(x)
        x = self.relu(x)
        x = self.pool(x)
        x = self.flatten(x)
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)  # logits for BCEWithLogitsLoss
        return x

model = HairTypeCNN().to(device)
model


HairTypeCNN(
  (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding=0, dilation=1, ceil_mode=False)
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (fc1): Linear(in_features=313632, out_features=64, bias=True)
  (fc2): Linear(in_features=64, out_features=1, bias=True)
  (relu): ReLU()
)

In [8]:
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.SGD(model.parameters(), lr=0.002, momentum=0.8)
criterion, optimizer


(BCEWithLogitsLoss(),
 SGD (
 Parameter Group 0
     dampening: 0
     differentiable: False
     foreach: None
     fused: None
     lr: 0.002
     maximize: False
     momentum: 0.8
     nesterov: False
     weight_decay: 0
 ))

In [9]:
num_epochs = 10
history = {'acc': [], 'loss': [], 'val_acc': [], 'val_loss': []}

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct_train = 0
    total_train = 0

    for images, labels in train_loader:
        images = images.to(device)
        labels = labels.float().unsqueeze(1).to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * images.size(0)
        # For binary classification with BCEWithLogitsLoss, apply sigmoid before thresholding for accuracy
        predicted = (torch.sigmoid(outputs) > 0.5).float()
        total_train += labels.size(0)
        correct_train += (predicted == labels).sum().item()

    epoch_loss = running_loss / len(train_dataset)
    epoch_acc = correct_train / total_train
    history['loss'].append(epoch_loss)
    history['acc'].append(epoch_acc)

    model.eval()
    val_running_loss = 0.0
    correct_val = 0
    total_val = 0

    with torch.no_grad():
        for images, labels in validation_loader:
            images = images.to(device)
            labels = labels.float().unsqueeze(1).to(device)

            outputs = model(images)
            loss = criterion(outputs, labels)

            val_running_loss += loss.item() * images.size(0)
            predicted = (torch.sigmoid(outputs) > 0.5).float()
            total_val += labels.size(0)
            correct_val += (predicted == labels).sum().item()

    val_epoch_loss = val_running_loss / len(validation_dataset)
    val_epoch_acc = correct_val / total_val
    history['val_loss'].append(val_epoch_loss)
    history['val_acc'].append(val_epoch_acc)

    print(
        f"Epoch {epoch + 1}/{num_epochs}, "
        f"Loss: {epoch_loss:.4f}, Acc: {epoch_acc:.4f}, "
        f"Val Loss: {val_epoch_loss:.4f}, Val Acc: {val_epoch_acc:.4f}"
    )


Epoch 1/10, Loss: 0.6462, Acc: 0.6362, Val Loss: 0.6032, Val Acc: 0.6517
Epoch 2/10, Loss: 0.5475, Acc: 0.7100, Val Loss: 0.7251, Val Acc: 0.6318
Epoch 3/10, Loss: 0.5533, Acc: 0.7250, Val Loss: 0.5991, Val Acc: 0.6716
Epoch 4/10, Loss: 0.4802, Acc: 0.7712, Val Loss: 0.6033, Val Acc: 0.6567
Epoch 5/10, Loss: 0.4334, Acc: 0.8025, Val Loss: 0.6196, Val Acc: 0.6766
Epoch 6/10, Loss: 0.3740, Acc: 0.8325, Val Loss: 0.7371, Val Acc: 0.6766
Epoch 7/10, Loss: 0.2721, Acc: 0.8838, Val Loss: 0.9223, Val Acc: 0.6418
Epoch 8/10, Loss: 0.2478, Acc: 0.9000, Val Loss: 0.7294, Val Acc: 0.7214
Epoch 9/10, Loss: 0.2075, Acc: 0.9200, Val Loss: 0.7523, Val Acc: 0.7015
Epoch 10/10, Loss: 0.1494, Acc: 0.9450, Val Loss: 0.7894, Val Acc: 0.7015


In [10]:
# Question 1: loss function used
q1_loss_function = 'nn.BCEWithLogitsLoss()'
print('Q1 - loss function:', q1_loss_function)

# Question 2: total number of parameters in the model
total_params = sum(p.numel() for p in model.parameters())
print('Q2 - total parameters:', total_params)


Q1 - loss function: nn.BCEWithLogitsLoss()
Q2 - total parameters: 20073473


In [11]:
acc_no_aug = np.array(history['acc'][:10])
loss_no_aug = np.array(history['loss'][:10])

train_acc_median = float(np.median(acc_no_aug))
train_loss_std = float(np.std(loss_no_aug))

print('Q3 - median training accuracy (no augmentation):', train_acc_median)
print('Q4 - std of training loss (no augmentation):', train_loss_std)


Q3 - median training accuracy (no augmentation): 0.8175
Q4 - std of training loss (no augmentation): 0.1589676198412134


In [12]:
augmented_train_transforms = transforms.Compose([
    transforms.Resize(image_size),
    transforms.RandomRotation(50),
    transforms.RandomResizedCrop(200, scale=(0.9, 1.0), ratio=(0.9, 1.1)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    ),
])

augmented_train_dataset = datasets.ImageFolder(train_dir, transform=augmented_train_transforms)
train_dataset = augmented_train_dataset
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
len(train_dataset)


800

In [13]:
num_epochs_aug = 10

for epoch in range(num_epochs_aug):
    model.train()
    running_loss = 0.0
    correct_train = 0
    total_train = 0

    for images, labels in train_loader:
        images = images.to(device)
        labels = labels.float().unsqueeze(1).to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * images.size(0)
        predicted = (torch.sigmoid(outputs) > 0.5).float()
        total_train += labels.size(0)
        correct_train += (predicted == labels).sum().item()

    epoch_loss = running_loss / len(train_dataset)
    epoch_acc = correct_train / total_train
    history['loss'].append(epoch_loss)
    history['acc'].append(epoch_acc)

    model.eval()
    val_running_loss = 0.0
    correct_val = 0
    total_val = 0

    with torch.no_grad():
        for images, labels in validation_loader:
            images = images.to(device)
            labels = labels.float().unsqueeze(1).to(device)

            outputs = model(images)
            loss = criterion(outputs, labels)

            val_running_loss += loss.item() * images.size(0)
            predicted = (torch.sigmoid(outputs) > 0.5).float()
            total_val += labels.size(0)
            correct_val += (predicted == labels).sum().item()

    val_epoch_loss = val_running_loss / len(validation_dataset)
    val_epoch_acc = correct_val / total_val
    history['val_loss'].append(val_epoch_loss)
    history['val_acc'].append(val_epoch_acc)

    print(
        f"[Aug] Epoch {epoch + 1}/{num_epochs_aug}, "
        f"Loss: {epoch_loss:.4f}, Acc: {epoch_acc:.4f}, "
        f"Val Loss: {val_epoch_loss:.4f}, Val Acc: {val_epoch_acc:.4f}"
    )


[Aug] Epoch 1/10, Loss: 0.7357, Acc: 0.5975, Val Loss: 0.6002, Val Acc: 0.6567
[Aug] Epoch 2/10, Loss: 0.5931, Acc: 0.6325, Val Loss: 0.5686, Val Acc: 0.7214
[Aug] Epoch 3/10, Loss: 0.5813, Acc: 0.6600, Val Loss: 0.5564, Val Acc: 0.7214
[Aug] Epoch 4/10, Loss: 0.5587, Acc: 0.7188, Val Loss: 0.6339, Val Acc: 0.6517
[Aug] Epoch 5/10, Loss: 0.5134, Acc: 0.7113, Val Loss: 0.5781, Val Acc: 0.7164
[Aug] Epoch 6/10, Loss: 0.5083, Acc: 0.7338, Val Loss: 0.6274, Val Acc: 0.6567
[Aug] Epoch 7/10, Loss: 0.5080, Acc: 0.7262, Val Loss: 0.4988, Val Acc: 0.7562
[Aug] Epoch 8/10, Loss: 0.5136, Acc: 0.7312, Val Loss: 0.5231, Val Acc: 0.7463
[Aug] Epoch 9/10, Loss: 0.5143, Acc: 0.7412, Val Loss: 0.6088, Val Acc: 0.6567
[Aug] Epoch 10/10, Loss: 0.5054, Acc: 0.7338, Val Loss: 0.5119, Val Acc: 0.7363


In [14]:
# Use only the epochs trained with augmentation (the last 10 entries)
val_loss_aug = np.array(history['val_loss'][-10:])
val_acc_aug = np.array(history['val_acc'][-10:])

test_loss_mean_aug = float(val_loss_aug.mean())
test_acc_mean_last5_aug = float(val_acc_aug[-5:].mean())

print('Q5 - mean test loss (with augmentation):', test_loss_mean_aug)
print('Q6 - avg test accuracy epochs 6-10 (with augmentation):', test_acc_mean_last5_aug)


Q5 - mean test loss (with augmentation): 0.5707233409976485
Q6 - avg test accuracy epochs 6-10 (with augmentation): 0.7104477611940299


In [15]:
print('Summary for homework submission:')
print('Q1: loss function -> nn.BCEWithLogitsLoss()')
print('Q2: total parameters ->', total_params)
print('Q3: median training accuracy (no aug) ->', train_acc_median)
print('Q4: std of training loss (no aug) ->', train_loss_std)
print('Q5: mean test loss (with aug) ->', test_loss_mean_aug)
print('Q6: avg test accuracy epochs 6-10 (with aug) ->', test_acc_mean_last5_aug)


Summary for homework submission:
Q1: loss function -> nn.BCEWithLogitsLoss()
Q2: total parameters -> 20073473
Q3: median training accuracy (no aug) -> 0.8175
Q4: std of training loss (no aug) -> 0.1589676198412134
Q5: mean test loss (with aug) -> 0.5707233409976485
Q6: avg test accuracy epochs 6-10 (with aug) -> 0.7104477611940299
